In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import skimage.io
import cv2

In [ ]:
curr_dir = os.getcwd()
path = os.path.dirname(curr_dir)
print(path)
csv_file_path = os.path.join(path, 'dataset/DataCSV/multicam_full_h36m.csv')

In [ ]:
df = pd.read_csv(csv_file_path)
df = df.drop(['Unnamed: 0','idx'], axis=1)
#df = df.sort_values(['score'],ascending=False)

In [ ]:
#df = df.drop_duplicates(subset = ['image_id','pos_class'], keep = 'first')
#df = df.sort_index()

In [ ]:
df['pos_class'].value_counts()

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
#df = df.drop(['nose_x','nose_y','LEye_x',
#       'LEye_y', 'REye_x', 'REye_y', 'LEar_x', 'LEar_y', 'REar_x', 'REar_y'], axis=1)

In [ ]:
# Load a random image from the images folder
#file_names = next(os.walk(IMAGE_DIR))[2]

#image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
image = skimage.io.imread('/home/rasho/Falling-Person-Detection-based-On-AlphaPose/input/Falling_Standing/FallingDown_01/00400.png')
image.shape

In [ ]:
box = df['box'][400]
box

In [ ]:
x = float(box.strip('[,]').split(',')[0])
y = float(box.strip('[,]').split(',')[1])
w = float(box.strip('[,]').split(',')[2])
h = float(box.strip('[,]').split(',')[3])

In [ ]:
plt.imshow(image)

In [ ]:
bbox = [x,x+w,y,y+h] #xmin,xmax,ymin,ymax
cv2.rectangle(image, (int(bbox[0]), int(bbox[2])), (int(bbox[1]),int(bbox[3])), (0,255), 3)
plt.imshow(image)

In [ ]:
import math
def get_angle(x,y):
    #return angle in degrees
    return (math.atan2(y,x)*(180/math.pi))

In [ ]:
df['foot_x'] = df['RFoot_x']-df['LFoot_x']
df['foot_y'] = df['RFoot_y']-df['LFoot_y']
df['angle_x']= df['Head_x']-df['foot_x']
df['angle_y']= df['Head_y']-df['foot_y']

In [ ]:
df[['angle_x','angle_y']]

In [ ]:
df['angle']= df.apply(lambda row: get_angle(x = row['angle_x'], y = row['angle_y']), axis=1)

In [ ]:
df.head()

In [ ]:
df3 = df[['pos_class','angle']]
falling_angle  = df3.loc[(df3['pos_class']=='Falling')]
standing_angle = df3.loc[(df3['pos_class']=='Standing')]
falling_angle  = falling_angle.iloc[:160].to_numpy()
standing_angle = standing_angle.iloc[:160].to_numpy()

In [ ]:
falling_angle

In [ ]:
falling_angle[:,1].min()

In [ ]:
falling_angle[:,1].max()

In [ ]:
falling_angle[:,1].mean()

In [ ]:
standing_angle

In [ ]:
standing_angle[:,1].min()

In [ ]:
standing_angle[:,1].max()

In [ ]:
standing_angle[:,1].mean()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
frames = np.arange(0,len(falling_angle))
plt.plot(frames,falling_angle[:,1],label='FallingDown')
plt.plot(frames,standing_angle[:,1],label='Standing')
plt.xlabel('Frames')
plt.ylabel('Angle in degrees')
plt.title('The angle between the center line of the Human and the Ground')
plt.legend()
plt.show()

In [ ]:
df4 = df[['pos_class','score']]
falling_dist = df4.loc[(df4['pos_class']=='Falling')]
standing_dist = df4.loc[(df4['pos_class']=='Standing')]
falling_dist  = falling_dist.iloc[:160].to_numpy()
standing_dist = standing_dist.iloc[:160].to_numpy()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
frames = np.arange(0,len(falling_dist))
plt.plot(frames,falling_dist[:,1],label='FallingDown')
plt.plot(frames,standing_dist[:,1],label='Standing')
plt.xlabel('Frames')
plt.ylabel('Score')
plt.title('Person Detection Confidence score')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
num_bins = 50
plt.hist(df4['score'],num_bins,density=True)
#plt.hist(stand[:,0],num_bins,density=True)
plt.show()

In [ ]:
df['score'].mean()

In [ ]:
df['score'].max()

In [ ]:
df['score'].min()

In [ ]:
plt.hist??

In [ ]:
df['Knee_x'] = df['RKnee_x']- df['LKnee_x']
df['Knee_y'] = df['RKnee_y']- df['LKnee_y']
df['SK_x']= df['Spine_x']-df['Knee_x']
df['SK_y']= df['Spine_y']-df['Knee_y']
df['dist'] = np.sqrt(np.square(df['SK_x'])+np.square(df['SK_y']))

In [ ]:
df4 = df[['pos_class','dist']]
falling_dist = df4.loc[(df4['pos_class']=='Falling')]
standing_dist = df4.loc[(df4['pos_class']=='Standing')]
falling_dist  = falling_dist.iloc[:160].to_numpy()
standing_dist = standing_dist.iloc[:160].to_numpy()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
frames = np.arange(0,len(falling_dist))
plt.plot(frames,falling_dist[:,1],label='FallingDown')
plt.plot(frames,standing_dist[:,1],label='Standing')
plt.xlabel('Frames')
plt.ylabel('Distance')
plt.title('Distance between Spine and Knee center')
plt.legend()
plt.show()

In [ ]:
df4['dist'].mean()

In [ ]:
df4['dist'].min()

In [ ]:
df4['dist'].max()

In [ ]:
#df4.loc[(df4['pos_class']=='NotFalls')&(df4['dist']<300)]
df4.loc[(df4['pos_class']=='Standing')&(df4['dist']<300)]

In [ ]:
def get_ratio(box):
    
    x = float(box.strip('[,]').split(',')[0])
    y = float(box.strip('[,]').split(',')[1])
    w = float(box.strip('[,]').split(',')[2])
    h = float(box.strip('[,]').split(',')[3])
    return w/h

In [ ]:
df['box_ratio']=df['box'].map(get_ratio)

In [ ]:
df['box_ratio'][5273]

In [ ]:
df5 = df[['pos_class','box_ratio']]
falling_ratio  = df5.loc[(df5['pos_class']=='Falling')]
standing_ratio = df5.loc[(df5['pos_class']=='Standing')]
falling_ratio  = falling_ratio.iloc[:3300].to_numpy()
standing_ratio = standing_ratio.iloc[:3300].to_numpy()

In [ ]:
falling_ratio[:,1].mean()

In [ ]:
falling_ratio[:,1].max()

In [ ]:
falling_ratio[:,1].min()

In [ ]:
standing_ratio[:,1].mean()

In [ ]:
standing_ratio[:,1].max()

In [ ]:
standing_ratio[:,1].min()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14,7))
frames = np.arange(0,len(falling_ratio ))
plt.plot(frames[:160],falling_ratio[:160,1],label='FallingDown')
plt.plot(frames[:160],standing_ratio[:160,1],label='Standing')
plt.xlabel('Frames')
plt.ylabel('Box W/H Ratio')
plt.title('Box W/H Ratio for Falling and Standing person')
plt.legend()
plt.show()

In [ ]:
df2 = df[['image_id','pos_class','box_ratio']]

In [ ]:
df2.loc[(df2['pos_class']=='FallingDown')&(df2['box_ratio']>2)]

In [ ]:
df2.loc[(df2['pos_class']=='FallingDown')&(df2['box_ratio']<2)]

In [ ]:
df2.loc[(df2['pos_class']=='Standing')&(df2['box_ratio']>2)]

In [ ]:
df2.loc[(df2['pos_class']=='FallingDown')& (df2['box_ratio']>0.95) | (df2['pos_class']=='Standing')& (df2['box_ratio']<0.95)]

In [ ]:
df2['pos_class'].unique

In [ ]:
x1_diff,y1_diff = feat_df['LShoulder_x']-feat_df['nose_x'],feat_df['LShoulder_y']-feat_df['nose_y']
x1y1_len = np.sqrt(np.square(x1_diff)+np.square(y1_diff))
x1,y1 = x1_diff/x1y1_len,y1_diff/x1y1_len
df['x1'] = x1
df['y1'] = y1

In [ ]:
x2_diff, y2_diff = feat_df['RShoulder_x']-feat_df['nose_x'],feat_df['RShoulder_y']-feat_df['nose_y']
x2y2_len = np.sqrt(np.square(x2_diff)+np.square(y2_diff))
x2, y2 = x2_diff/x2y2_len, y2_diff/x2y2_len
df['x2'] = x2
df['y2'] = y2

In [ ]:
df.columns

In [ ]:
df

In [ ]:
df[df.iloc[:,30:].isna().any(axis=1)]

In [ ]:
df = df.dropna()

In [ ]:
df.isna().any()

In [ ]:
df.to_csv('data/victorized_res.csv')

In [ ]:
curr_dir = os.getcwd()
csv_file_path = os.path.join(curr_dir, 'data/res_with_flips.csv')

In [ ]:
df2 = pd.read_csv(csv_file_path)
df2 = df2.drop(['Unnamed: 0','idx'], axis=1)
df2 = df2.sort_values(['score'],ascending=False)

In [ ]:
df2 = df2.drop_duplicates(subset = ['image_id','pos_class'], keep = 'first')
df2 = df2.sort_index()
df2.head()

In [ ]:
df2

In [ ]:
df2.to_csv('data/clean_res_with_flips.csv')

In [ ]:
df.columns

In [ ]:
def get_center(coord1, coord2):
    return (coord1+coord2)/2

In [ ]:
# get center of Right Hip joint and Left Hip joint
df['HipCent_x'] = df.apply(lambda row: get_center(coord1=row['RHip_x'],coord2=row['LHip_x']), axis=1)
df['HipCent_y'] = df.apply(lambda row: get_center(coord1=row['RHip_y'],coord2=row['LHip_y']), axis=1)

In [ ]:
df.head()

In [ ]:
def get_vector(p1_x,p1_y,p2_x,p2_y):
    
    return p2_x-p1_x, p2_y-p1_y

In [ ]:
#get waist, Spine2, Right thigh, Left Thigh, Right Calf, Left Calf vectors.
df['waist_x'], df['waist_y'] = zip(*df.apply(lambda row: get_vector(p1_x=row['RHip_x'],p1_y=row['RHip_y'],p2_x=row['LHip_x'],p2_y=row['LHip_y']), axis=1))
df['Spine2_x'], df['Spine2_y'] = zip(*df.apply(lambda row: get_vector(p1_x=row['Thorax_x'],p1_y=row['Thorax_y'],p2_x=row['HipCent_x'],p2_y=row['HipCent_y']), axis=1))
df['Rthigh_x'], df['Rthigh_y'] = zip(*df.apply(lambda row: get_vector(p1_x=row['RHip_x'],p1_y=row['RHip_y'],p2_x=row['RKnee_x'],p2_y=row['RKnee_y']), axis=1))
df['Lthigh_x'], df['Lthigh_y'] = zip(*df.apply(lambda row: get_vector(p1_x=row['LHip_x'],p1_y=row['LHip_y'],p2_x=row['LKnee_x'],p2_y=row['LKnee_y']), axis=1))
df['Rcalf_x'], df['Rcalf_y'] = zip(*df.apply(lambda row: get_vector(p1_x=row['RKnee_x'],p1_y=row['RKnee_y'],p2_x=row['RFoot_x'],p2_y=row['RFoot_y']), axis=1))
df['Lcalf_x'], df['Lcalf_y'] = zip(*df.apply(lambda row: get_vector(p1_x=row['LKnee_x'],p1_y=row['LKnee_y'],p2_x=row['LFoot_x'],p2_y=row['LFoot_y']), axis=1))


In [ ]:
def get_vectorRatio(p1_x,p1_y,p2_x,p2_y):
    # add small number to numerator and denominator
    p1_len = np.sqrt(np.square(p1_x)+np.square(p1_y))
    p2_len = np.sqrt(np.square(p2_x)+np.square(p2_y))
    return (p1_len+0.00001)/(p2_len+0.00001)

In [ ]:
df['SRatio'] = df.apply(lambda row: get_vectorRatio(p1_x=row['Spine2_x'],p1_y=row['Spine2_y'],p2_x=row['waist_x'],p2_y=row['waist_y']), axis=1)

In [ ]:
df2 = df[['SRatio','pos_class']]

In [ ]:
fall = df2.loc[(df2['pos_class']=='Falling')]
fall = fall.iloc[:4000].iloc[:,:1].to_numpy()

In [ ]:
fall

In [ ]:
stand = df2.loc[(df2['pos_class']=='Standing')]
stand = stand[:4000].iloc[:,:1].to_numpy()

In [ ]:
stand

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14,7))
frames = np.arange(0,len(fall))
plt.plot(frames[:160],fall[:160,0],label='FallingDown')
plt.plot(frames[:160], stand[:160,0],label='Standing')
plt.xlabel('Frames')
plt.ylabel('Spine Ratio')
plt.title('Spine Ratio for Standing and Falling')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
num_bins = 5
plt.hist(fall[:,0],num_bins,density=True)
plt.hist(stand[:,0],num_bins,density=True)
plt.show()

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df['hip_v'] = df['HipCent_y'].diff(periods=5).abs()/4

In [ ]:
df['spine_v'] = df['Spine_y'].diff(periods=5).abs()/4

In [ ]:
df['head_v'] = df['Head_y'].diff(periods=5).abs()/4

In [ ]:
df.isna().any()

In [ ]:
df[df.iloc[:,30:].isna().any(axis=1)]

In [ ]:
df = df.dropna()

In [ ]:
df.isna().any()

In [ ]:
df5 = df[['pos_class','hip_v','spine_v','head_v']]
fall = df5.loc[(df5['pos_class']=='Falling')]
stand = df5.loc[(df5['pos_class']=='Standing')]
fall  = fall.iloc[:3300].to_numpy()
stand  = stand.iloc[:3300].to_numpy()

In [ ]:
fall[:,1].min()

In [ ]:
fall[:,1].mean()

In [ ]:
fall[:,1].max()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14,7))
frames = np.arange(0,len(fall))
plt.plot(frames[:160],fall[:160,1],label='FallingDown')
plt.plot(frames[:160], stand[:160,1],label='Standing')
plt.xlabel('Frames')
plt.ylabel('Hip Speed')
plt.title('Hip Speed for Standing and Falling')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14,7))
frames = np.arange(0,len(fall))
plt.plot(frames[:160],fall[:160,2],label='FallingDown')
plt.plot(frames[:160], stand[:160,2],label='Standing')
plt.xlabel('Frames')
plt.ylabel('Spine Speed')
plt.title('Spine Speed for Standing and Falling')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(14,7))
frames = np.arange(0,len(fall))
plt.plot(frames[:160],fall[:160,3],label='FallingDown')
plt.plot(frames[:160], stand[:160,3],label='Standing')
plt.xlabel('Frames')
plt.ylabel('Head Speed')
plt.title('Head Speed for Standing and Falling')
plt.legend()
plt.show()

In [ ]:
n_frames = 5
pose2d_size =34
humanData = np.zeros([n_frames, pose2d_size])

In [ ]:
humanData

In [ ]:
pose = np.ones(34)

In [ ]:
humanData[0,:]= pose

In [ ]:
humanData

In [ ]:
humanData[1:5,:]